快速使用

In [1]:
#首先，您需要导入fluid库
import paddle.fluid as fluid

1.使用Fluid创建5个元素的一维数组，其中每个元素都为1

In [2]:
# 定义数组维度及数据类型，可以修改shape参数定义任意大小的数组
data = fluid.layers.ones(shape=[5], dtype='int64')
# 在CPU上执行运算
place = fluid.CPUPlace()
# 创建执行器
exe = fluid.Executor(place)
# 执行计算
ones_result = exe.run(fluid.default_main_program(),
                    # 获取数据data
                    fetch_list=[data],
                    return_numpy=True)
# 输出结果
print(ones_result[0])

[1 1 1 1 1]


2.使用Fluid将两个数组按位相加

In [4]:
# 调用 elementwise_op 将生成的一维数组按位相加
add = fluid.layers.elementwise_add(data,data)
# 定义运算场所
place = fluid.CPUPlace()
exe = fluid.Executor(place)
# 执行计算
add_result = exe.run(fluid.default_main_program(),
                    fetch_list=[add],
                    return_numpy=True)
# 输出结果
print (add_result[0])

[2 2 2 2 2]


3.使用Fluid转换数据类型

In [5]:
# 将一维整型数组，转换成float64类型
cast = fluid.layers.cast(x=data, dtype='float64')
# 定义运算场所执行计算
place = fluid.CPUPlace()
exe = fluid.Executor(place)
cast_result = exe.run(fluid.default_main_program(),
                 fetch_list=[cast],
                 return_numpy=True)
# 输出结果
print(cast_result[0])

[1. 1. 1. 1. 1.]


运行线性回归模型

通过上面的小例子，相信您已经对如何使用Fluid操作数据有了一定的了解，那么试着创建一个test.py，并粘贴下面的代码吧。

这是一个简单的线性回归模型，来帮助我们快速求解4元一次方程。

In [10]:
#加载库
import paddle.fluid as fluid
import numpy as np
#生成数据
np.random.seed(0)#当我们设置相同的seed，每次生成的随机数相同。
outputs = np.random.randint(5, size=(10, 4))
print(outputs)
"""
函数的作用是，返回一个随机整型数，范围从低（包括）到高（不包括），即[low, high)。
如果没有写参数high的值，则返回[0,low)的值。
"""
res = []
for i in range(10):
    # 假设方程式为 y=4a+6b+7c+2d
    y = 4*outputs[i][0]+6*outputs[i][1]+7*outputs[i][2]+2*outputs[i][3]
    res.append([y])
    print(res)

[[4 0 3 3]
 [3 1 3 2]
 [4 0 0 4]
 [2 1 0 1]
 [1 0 1 4]
 [3 0 3 0]
 [2 3 0 1]
 [3 3 3 0]
 [1 1 1 0]
 [2 4 3 3]]
[[43]]
[[43], [43]]
[[43], [43], [24]]
[[43], [43], [24], [16]]
[[43], [43], [24], [16], [19]]
[[43], [43], [24], [16], [19], [33]]
[[43], [43], [24], [16], [19], [33], [28]]
[[43], [43], [24], [16], [19], [33], [28], [51]]
[[43], [43], [24], [16], [19], [33], [28], [51], [17]]
[[43], [43], [24], [16], [19], [33], [28], [51], [17], [59]]


In [11]:
# 定义数据
train_data = np.array(outputs).astype('float32')
print(train_data)
y_true = np.array(res).astype('float32')
print(y_true)

[[4. 0. 3. 3.]
 [3. 1. 3. 2.]
 [4. 0. 0. 4.]
 [2. 1. 0. 1.]
 [1. 0. 1. 4.]
 [3. 0. 3. 0.]
 [2. 3. 0. 1.]
 [3. 3. 3. 0.]
 [1. 1. 1. 0.]
 [2. 4. 3. 3.]]
[[43.]
 [43.]
 [24.]
 [16.]
 [19.]
 [33.]
 [28.]
 [51.]
 [17.]
 [59.]]


In [14]:
#定义网络
x = fluid.layers.data(name='x', shape=[4], dtype='float32')
y = fluid.layers.data(name='y', shape=[1], dtype='float32')
y_predict = fluid.layers.fc(input=x, size=1, act=None)

In [15]:
#定义损失函数
cost = fluid.layers.square_error_cost(input=y_predict,label=y)
avg_cost = fluid.layers.mean(cost)

In [16]:
#定义优化方法
sgd_optimizer = fluid.optimizer.SGD(learning_rate=0.05)
sgd_optimizer.minimize(avg_cost)

([inputs {
    parameter: "Grad"
    arguments: "fc_0.b_0@GRAD"
  }
  inputs {
    parameter: "LearningRate"
    arguments: "learning_rate_0"
  }
  inputs {
    parameter: "Param"
    arguments: "fc_0.b_0"
  }
  outputs {
    parameter: "ParamOut"
    arguments: "fc_0.b_0"
  }
  type: "sgd"
  attrs {
    name: "op_role"
    type: INT
    i: 2
  }
  attrs {
    name: "op_role_var"
    type: STRINGS
    strings: "fc_0.b_0"
    strings: "fc_0.b_0@GRAD"
  }
  attrs {
    name: "op_namescope"
    type: STRING
    s: "/optimizer/"
  }
  attrs {
    name: "op_callstack"
    type: STRINGS
    strings: "  File \"D:\\Anaconda3\\lib\\site-packages\\paddle\\fluid\\framework.py\", line 1654, in append_op\n    attrs=kwargs.get(\"attrs\", None))\n"
    strings: "  File \"D:\\Anaconda3\\lib\\site-packages\\paddle\\fluid\\optimizer.py\", line 547, in _append_optimize_op\n    stop_gradient=True)\n"
    strings: "  File \"D:\\Anaconda3\\lib\\site-packages\\paddle\\fluid\\optimizer.py\", line 296, in _cre

In [22]:
#参数初始化
cpu = fluid.CPUPlace()
exe = fluid.Executor(cpu)
exe.run(fluid.default_startup_program())
##开始训练，迭代500次
for i in range(500):
    outs = exe.run(
             feed={'x':train_data,'y':y_true},
             fetch_list=[y_predict.name, avg_cost.name])
    if i % 50 == 0:
        print(outs)
        print('iter={:.0f}, cost={}'.format(i,outs[1][0]))

[array([[ 1.4025782 ],
       [ 0.6977779 ],
       [ 1.7664022 ],
       [ 0.08172041],
       [ 2.1145957 ],
       [-0.11558425],
       [-0.44467908],
       [-0.90518343],
       [-0.30172783],
       [ 0.54314005]], dtype=float32), array([1286.9109], dtype=float32)]
iter=0, cost=1286.910888671875
[array([[42.74027 ],
       [42.957428],
       [23.737972],
       [16.397247],
       [19.521908],
       [33.268154],
       [28.118856],
       [50.850567],
       [17.725779],
       [58.724655]], dtype=float32), array([0.12790576], dtype=float32)]
iter=50, cost=0.12790575623512268
[array([[42.835995],
       [42.97896 ],
       [23.813679],
       [16.240324],
       [19.35332 ],
       [33.170025],
       [28.059565],
       [50.898888],
       [17.464321],
       [58.84046 ]], dtype=float32), array([0.05283729], dtype=float32)]
iter=100, cost=0.05283728986978531
[array([[42.894466],
       [42.986526],
       [23.87987 ],
       [16.154526],
       [19.227621],
       [33.109417]

In [23]:
#存储训练结果
params_dirname = 'result'
fluid.io.save_inference_model(params_dirname, ['x'], [y_predict], exe)

['save_infer_model/scale_0']

In [25]:
# 开始预测
infer_exe = fluid.Executor(cpu)
inference_scope = fluid.Scope()
# 加载训练好的模型
with fluid.scope_guard(inference_scope):
    [inference_program, feed_target_names,
    fetch_targets] = fluid.io.load_inference_model(params_dirname,infer_exe)
    
# 生成测试数据
test = np.array([[[9],[5],[2],[10]]]).astype('float32')
# 进行预测
results = infer_exe.run(inference_program, feed={'x':test},
                       fetch_list = fetch_targets)
# 给出题目为 【9,5,2,10】 输出y=4*9+6*5+7*2+10*2的值
print(results)
print ("9a+5b+2c+10d={}".format(results[0][0]))

[array([[99.94476]], dtype=float32)]
9a+5b+2c+10d=[99.94476]
